In [25]:
import numpy as np
import tensorflow
from tensorflow.keras.datasets import mnist

np.set_printoptions(threshold=np.inf)

### Define some functions for converting from label to one hot encoding and vice versa:

In [26]:
def labels_to_onehotvector(labels):
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    onehotvector = np.zeros((len(labels), len(unique_labels)))
    for index, label in enumerate(labels):
        onehotvector[int(index), int(label)] = int(1)
    return onehotvector

In [27]:
def onehotvectors_to_labels(onehotvectors):
    labels = np.zeros(onehotvectors.shape[0])
    labels = onehotvectors.argmax(axis=1)
    return labels

### Next we define some functions for better readability in the later parts. First is the ReLU function for the hidden layer activations
ReLU(z) = max(0, z)

In [28]:
def relu(Z):
    return np.maximum(0,Z)

### ... and a function for getting the gradient of the ReLU activations at the hidden layers
d ReLU(Z)/dz = 1 if Z>0 ; 0 otherwise

In [29]:
def delta_relu(Z):
    return (Z>0).astype(int)

### Next is a function for the output layer activation: Softmax:
softmax(Z) = exp(Z)/(SUM(exp(Z))

To avoid python returning NaN due to exceeding of the max float64 value, we multiply a constant C to both the numerator and denominator. Setting the C to be the negative maximum value among all the softmax inputs, we have...

stable_softmax(Z) = exp(Z+max(Z))/(SUM(exp(Z+max(Z))

In [30]:
def softmax(Z):
    exps = np.exp(Z - np.max(Z))
    exps = exps / np.sum(exps)
    return exps

### ... and a function for getting the gradient of the Softmax activation at the output layer, to be used in during backpropagation later on

d softmax(Z)/dZ = Z - y

where y = one-hot encoded labels

In [31]:
def delta_softmax(softmax_out, onehot_true_labels):
    return softmax_out - onehot_true_labels

### We define a function for computing the categorical cross-entropy Loss of the whole network: L
L = -SUM(y*log(y_pred))

where **y** is the one-hot encoded true labels and **y_pred** is the prediction of the network (i.e. output of the softmax output activation layer)

***To save on computation time:***
Since **y** are in one-hot encoded format, we can instead compute the -log(y_pred) only at the column indices that are '1' and then proceed to computing the summation. This is much faster compared to matrix multiplication

In [32]:
def cross_entropy(y_pred, y):
    #log_likelihood = -np.multiply(y,np.log(y_pred)) #too slow
    log_likelihood = -np.log(y_pred[y.argmax()])
    
    #print('\ty', np.transpose(y))
    #print('y_pred', np.transpose(y_pred))
    #print('log_likelihood', np.transpose(log_likelihood))
    loss = np.sum(log_likelihood)
                             
    return loss

### ... and its corresponding gradient:
dL/dy_pred = y_pred - y

where once again, **y** is the one-hot encoded true labels and **y_pred** is the prediction of the network (i.e. output of the softmax output activation layer)

For computation speed, compute for y_pred - 1 only at the index where y=1 (e.g. y.argmax()

In [33]:
def delta_cross_entropy(y_pred, y):
    y_pred[y.argmax()] = y_pred[y.argmax()] - 1
    delta = y_pred
    
    return delta

### Next we define a method for initializing the network weights and biases:
For the initialization, we use the default setting of keras **Dense** class:

Weight initialization: Glorot Uniform: (-6/sqrt(m+n) , 6/sqrt(m+n))
where m=number of inputs, n=number of outputs

Bias initialization: all zeros

In [34]:
def init_neurons(num_input, num_hidden1_neurons, num_hidden2_neurons, num_output):
    #INPUT LAYER
    x_in = np.zeros((num_input, 1))
    init = 0.1
    #HIDDEN LAYER 1: num_input inputs, num_hidden1_neurons outputs
    #init = 6/np.sqrt(num_input+num_hidden1_neurons)
    w_h1 = np.random.uniform(low=-init, high=init, size=(num_hidden1_neurons, num_input))
    b_h1 = np.zeros([num_hidden1_neurons, 1])

    #HIDDEN LAYER 2: num_hidden1_neurons inputs, num_hidden2_neurons outputs
    #init = 6/np.sqrt(num_hidden1_neurons+num_hidden2_neurons)
    w_h2 = np.random.uniform(low=-init, high=init, size=(num_hidden2_neurons, num_hidden1_neurons))
    b_h2 = np.zeros([num_hidden2_neurons, 1])

    #OUTPUT LAYER: num_hidden2_neurons inputs, num_output outputs
    #init = 6/np.sqrt(num_hidden2_neurons+num_output)
    w_out = np.random.uniform(low=-init, high=init, size=(num_output, num_hidden2_neurons))
    b_out = np.random.uniform(low=-init, high=init, size=(num_output, 1))
    
    return x_in, w_h1, b_h1, w_h2, b_h2, w_out, b_out

### We also define a method for generating data and their corresponding labels for getting a batch:

In [35]:
def batch(X, Y, batch_size):
    length = X.shape[0]
    for i in np.arange(0, length, batch_size):
        yield (X[i:min(i+batch_size, length)], Y[i:min(i+batch_size, length)])

### Method for forward-pass yielding the prediction **y_pred**

In [36]:
def predict(input_data, w_h1, b_h1, w_h2, b_h2, w_out, b_out):
    ## HIDDEN LAYER 1
    z_h1 = np.dot(w_h1,input_data) + b_h1
    a_h1 = relu(z_h1)
    #print("w_h1.shape", w_h1.shape, "input_data.shape", input_data.shape, "a_h1.shape", a_h1.shape)
    ## HIDDEN LAYER 2
    z_h2 = np.dot(w_h2, a_h1) + b_h2
    a_h2 = relu(z_h2)
    #print("w_h2.shape", w_h2.shape, "a_h1.shape", a_h1.shape, "a_h2.shape", a_h1.shape)
    ## OUTPUT LAYER
    z_out = np.dot(w_out, a_h2) + b_out
    #print("\n\tz_out=", np.transpose(z_out))
    y_pred = softmax(z_out)
    #print("\ty_pred=", np.transpose(y_pred))
    #print("w_out.shape", w_out.shape, "a_h2.shape", a_h2.shape, "y_pred.shape", y_pred.shape)
    
    return y_pred, a_h1, a_h2

In [37]:
def predict_batch(x_batch, w_h1, b_h1, w_h2, b_h2, w_out, b_out, y_batch):
    num_batches_processed = 0.0
    y_preds = np.zeros(y_batch.shape)
    # initialize the total error
    average_batch_error = 0.0
    total_batch_delta_out = 0.0
    for train_index, x in enumerate(x_batch): #SINGLE-BATCH LOOP
        x = x.reshape(-1, 1)
        y = y_batch[train_index].reshape(-1,1)

        ##### FORWARD PASS #####
        y_pred, a_h1, a_h2 = predict(x, w_h1, b_h1, w_h2, b_h2, w_out, b_out)

        ##### BATCH ERROR COMPUTATIONS ######
        error = cross_entropy(y_pred, y)
        #print('individual error:', error)
        average_batch_error = average_batch_error + error
        
        total_batch_delta_out = total_batch_delta_out+ delta_cross_entropy(y_pred, y)

        num_batches_processed += 1
        
        y_preds[train_index:] = np.transpose(y_pred) # collate the predictions
        last_x = x # to be returned for parameter updates after batch gradient computations
    
    average_batch_error = average_batch_error/x_batch.shape[0]
    total_batch_delta_out = total_batch_delta_out/x_batch.shape[0]
    
    return average_batch_error, total_batch_delta_out, num_batches_processed, last_x, y_preds, a_h1, a_h2
    

### Create a method for learning rate scheduling:

In [38]:
def get_LR(current_error, initial_LR):
    LR = initial_LR
    if current_error>0.006 and current_error<0.014:
        LR = initial_LR/2
    
    return LR

### Declare the weights and biases as globally shared parameters for ease of testing

In [39]:
w_h1, b_h1, w_h2, b_h2, w_out, b_out = np.zeros(6)

### Define a method for the training the artificial neural network: ann_fit()

In [40]:
def ann_fit(train_data, train_labels, num_input, num_hidden1_neurons, num_hidden2_neurons, num_output,
            batch_size=128, LR=0.1, validation_period=10, validation_data=None, validation_labels=None):
    global x_in, w_h1, b_h1, w_h2, b_h2, w_out, b_out 
    
    # Initialize the input, output, and hidden layer neurons (i.e. their weights and biases matrices)
    x_in, w_h1, b_h1, w_h2, b_h2, w_out, b_out = init_neurons(num_input, num_hidden1_neurons,
                                                            num_hidden2_neurons, num_output)
    # Remember the Initial LR for LR scheduling purposes:
    initial_LR = LR
    
    MAX_EPOCH=1000
    
    # Generate a vector for the epoch numbers
    epochs = range(0, MAX_EPOCH)
    
    # Stop training if the current total training error goes below this value
    ERR_TERMINATION_COND = 0.001
    
    total_error = 0.0
    num_batches_processed = 0.0
    
    for epoch_index in epochs:
        print('\n=========================================================\nEPOCH # %d' % (epoch_index+1))
        randomized_train_indices = np.random.permutation(train_data.shape[0])
        randomized_x_train = np.take(x_train, randomized_train_indices, axis=0);
        randomized_y_train = np.take(y_train, randomized_train_indices, axis=0);
        for x_batch, y_batch in batch(randomized_x_train, randomized_y_train, batch_size):
            #for train_index, x in enumerate(x_batch): #SINGLE-BATCH LOOP
            #    x = x.reshape(-1, 1)
            #    y = y_batch[train_index].reshape(-1,1)
            #    
            #    ##### FORWARD PASS #####
            #    y_pred, a_h1, a_h2 = predict(x, w_h1, b_h1, w_h2, b_h2, w_out, b_out)
            #    
            #    ##### BATCH ERROR COMPUTATIONS ######
            #    total_batch_error = total_batch_error + cross_entropy(y_pred, y)
            #    total_batch_delta_out = total_batch_delta_out+ delta_cross_entropy(y_pred, y)
            #    
            #    num_batches_processed += 1
            
            average_batch_error, total_batch_delta_out, temp_num_batches_processed, x, _, a_h1, a_h2 = predict_batch(x_batch, w_h1, b_h1, w_h2, b_h2, w_out, b_out, y_batch)
            total_error = total_error + average_batch_error
            num_batches_processed += temp_num_batches_processed
            
            ##### BACK PROPAGATION (PERFORMED AT AFTER EACH BATCH PROCESSING #####
            delta_h2 = delta_relu(a_h2)*(np.dot(np.transpose(w_out),total_batch_delta_out))
            delta_h1 = delta_relu(a_h1)*(np.dot(np.transpose(w_h2), delta_h2))

            ## Update the weights and biases
            w_out = w_out - LR*total_batch_delta_out*np.transpose(a_h2)
            b_out = b_out - LR*total_batch_delta_out

            w_h2 = w_h2 - LR*delta_h2*np.transpose(a_h1)
            b_h2 = b_h2 - LR*delta_h2

            w_h1 = w_h1 - LR*delta_h1*np.transpose(x)
            b_h1 = b_h1 - LR*delta_h1
        total_average_error = total_error/num_batches_processed
        print("\ttotal average error: %f @ LR=%f" % (total_error/num_batches_processed, LR))
        LR = get_LR(total_average_error, initial_LR)
        if( total_average_error < ERR_TERMINATION_COND):
            print('TRAINING ERROR TARGET REACHED! STOPPING TRAINING...')
            break
                
    

### Load the data and reshape the training and test data to 1x(28*28), then normalize:

In [41]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train = np.reshape(x_train, [-1, x_train.shape[1]*x_train.shape[2]])
x_train = x_train.astype('float64')/np.max(x_train)
x_test = np.reshape(x_test, [-1, x_test.shape[1]*x_test.shape[2]])
x_test = x_test.astype('float64')/np.max(x_test)

print('new x_train shape after reshaping: ', x_train.shape)
print('new x_test shape after reshaping: ', x_test.shape)

num_labels = len(np.unique(y_train))
y_train = labels_to_onehotvector(y_train)
y_test = labels_to_onehotvector(y_test)
print('new y_train shape after onehot vector encoding: ', y_train.shape)
print('new y_test shape after onehot vector encoding: ', y_test.shape)

x_train shape:  (60000, 28, 28)
y_train shape:  (60000,)
x_test shape:  (10000, 28, 28)
y_test shape:  (10000,)
new x_train shape after reshaping:  (60000, 784)
new x_test shape after reshaping:  (10000, 784)
new y_train shape after onehot vector encoding:  (60000, 10)
new y_test shape after onehot vector encoding:  (10000, 10)


# Main training loop

In [42]:
# Define number of neurons per layer
NUM_INPUT = x_train.shape[1]
NUM_HIDDEN1_NEURONS = 256
NUM_HIDDEN2_NEURONS = 256
NUM_OUTPUT = y_train.shape[1]

# Initial Learning Rate of Keras SGD Optimizer: https://keras.io/api/optimizers/sgd/
LR = 0.2

# Define the maximum number of epochs to run before stopping even if the stopping criteria is not met
MAX_EPOCH = 30000

ann_fit(x_train, y_train, num_input=NUM_INPUT, num_hidden1_neurons=NUM_HIDDEN1_NEURONS,
            num_hidden2_neurons=NUM_HIDDEN2_NEURONS, num_output=NUM_OUTPUT,
            batch_size=128, LR=LR)


EPOCH # 1
	total average error: 0.017414 @ LR=0.200000

EPOCH # 2
	total average error: 0.017148 @ LR=0.200000

EPOCH # 3
	total average error: 0.016892 @ LR=0.200000

EPOCH # 4
	total average error: 0.016717 @ LR=0.200000

EPOCH # 5
	total average error: 0.016389 @ LR=0.200000

EPOCH # 6
	total average error: 0.016031 @ LR=0.200000

EPOCH # 7
	total average error: 0.015628 @ LR=0.200000

EPOCH # 8
	total average error: 0.015306 @ LR=0.200000

EPOCH # 9
	total average error: 0.014976 @ LR=0.200000

EPOCH # 10
	total average error: 0.014724 @ LR=0.200000

EPOCH # 11
	total average error: 0.014508 @ LR=0.200000

EPOCH # 12
	total average error: 0.014277 @ LR=0.200000

EPOCH # 13
	total average error: 0.014077 @ LR=0.200000

EPOCH # 14
	total average error: 0.013982 @ LR=0.200000

EPOCH # 15
	total average error: 0.013925 @ LR=0.100000

EPOCH # 16
	total average error: 0.013862 @ LR=0.100000

EPOCH # 17
	total average error: 0.013801 @ LR=0.100000

EPOCH # 18
	total average error: 0.0137

	total average error: 0.009138 @ LR=0.100000

EPOCH # 73
	total average error: 0.009092 @ LR=0.100000

EPOCH # 74
	total average error: 0.009051 @ LR=0.100000

EPOCH # 75
	total average error: 0.009009 @ LR=0.100000

EPOCH # 76
	total average error: 0.008963 @ LR=0.100000

EPOCH # 77
	total average error: 0.008918 @ LR=0.100000

EPOCH # 78
	total average error: 0.008872 @ LR=0.100000

EPOCH # 79
	total average error: 0.008826 @ LR=0.100000

EPOCH # 80
	total average error: 0.008784 @ LR=0.100000

EPOCH # 81
	total average error: 0.008740 @ LR=0.100000

EPOCH # 82
	total average error: 0.008695 @ LR=0.100000

EPOCH # 83
	total average error: 0.008654 @ LR=0.100000

EPOCH # 84
	total average error: 0.008616 @ LR=0.100000

EPOCH # 85
	total average error: 0.008575 @ LR=0.100000

EPOCH # 86
	total average error: 0.008534 @ LR=0.100000

EPOCH # 87
	total average error: 0.008492 @ LR=0.100000

EPOCH # 88
	total average error: 0.008452 @ LR=0.100000

EPOCH # 89
	total average error: 0.008416 

KeyboardInterrupt: 

In [43]:
average_batch_error, _, _, _, y_pred, _, _ = predict_batch(x_test, w_h1, b_h1, w_h2, b_h2, w_out, b_out, y_test)

In [44]:
average_batch_error

0.6325183847403039

In [45]:
y_test[0], y_pred[0]

(array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]),
 array([ 0.00025296,  0.00021894,  0.00048948,  0.00037947,  0.00038958,
         0.00402058,  0.00067529, -0.01190333,  0.00014742,  0.0053296 ]))

In [46]:
### NOTE TO SELF: up next is: dropout, figure out what's wrong with SGD (too sloooow)